## Installation of libraries

In [2]:
!pip install --quiet transformers==4.5.0
!pip install --quiet sentencepiece==0.1.95
!pip install --quiet textwrap3==0.9.2
!pip install --quiet nltk==3.2.5

     |████████████████████████████████| 2.1 MB 3.1 MB/s 
     |████████████████████████████████| 895 kB 10.7 MB/s 
     |████████████████████████████████| 3.3 MB 33.5 MB/s 
     |████████████████████████████████| 1.2 MB 5.4 MB/s 


In [3]:
!pip install --quiet ipython-autotime
%load_ext autotime

time: 131 µs (started: 2022-04-27 14:45:09 +00:00)


In [8]:
from textwrap3 import wrap

text = ["""Equitable access to safe and effective vaccines is critical to ending the COVID-19 pandemic, so it is hugely encouraging to see so many vaccines proving and going into development. WHO is working tirelessly with partners to develop, manufacture and deploy safe and effective vaccines. 

Safe and effective vaccines are a game-changing tool: but for the foreseeable future we must continue wearing masks, cleaning our hands, ensuring good ventilation indoors, physically distancing and avoiding crowds. 

Being vaccinated does not mean that we can throw caution to the wind and put ourselves and others at risk, particularly because research is still ongoing into how much vaccines protect not only against disease but also against infection and transmission.

See WHO’s landscape of COVID-19 vaccine candidates for the latest information on vaccines in clinical and pre-clinical development, generally updated twice a week. WHO’s COVID-19 dashboard, updated daily, also features the number of vaccine doses administered globally, with more detail provided on the dedicated COVID-19 vaccination dashboard. At a regional level, there is an AFRO COVID-19 vaccines dashboard and a PAHO COVID-19 vaccines deliveries dashboard. 

But it’s not vaccines that will stop the pandemic, it’s vaccination. We must ensure fair and equitable access to vaccines, and ensure every country receives them and can roll them out to protect their people, starting with the most vulnerable.""", """Worried about the rising COVID cases in the national capital, traders’ associations across Delhi have resumed regular sanitisation, put up posters of dos and don’ts, and are ensuring proper masking in the markets.

The national capital has been witnessing an uptick in coronavirus infections over the past few days. The number of active cases has now increased to 4,168 from 601 on April 11, according to health department data.

The hospitalisation rate so far has been low, accounting for less than three per cent of the total active cases, it said.""", """With booster doses of COVID-19 available for all adults, the traders’ associations have also been planning for vaccine camps to ensure the safety of shop owners and customers.

The New Delhi Traders Association (NDTA) has said it is ensuring that all necessary precautions are followed in the markets and have urged all its members to take booster doses.

Atul Bhargava, the president of NDTA, said the over 400 members of the association have been asked to remain cautious.""", """Traders at Chandni Chowk, the area which is witnessing a surge in footfall due to the wedding season, have brought back COVID protocols and are ensuring that masking is followed.

Sanjay Bhargava, president of Chandni Chowk Vyapar Mandal, told PTI they would be issuing a new circular in the coming days regarding COVID 19. During the omicron wave earlier this year, city authorities had brought back the night curfew, and had forced markets to follow the odd-even arrangement for several weeks.""", """Along with the survey, the state government will continue with its search to find out whether any new variant of coronavirus has evolved in the state, he added. 

Sentinel surveillance is monitoring of the rate of occurrence of a specific disease through the voluntary network of doctors and laboratories aiming to assess the stability in health levels of the population of a particular region."""]

print(len(text))
for i in  range(len(text)):
  print('Text', i+1, ':')
  for wrp in wrap(text[i], 150):
    print (wrp)
  print ("\n")

5
Text 1 :
Equitable access to safe and effective vaccines is critical to ending the COVID-19 pandemic, so it is hugely encouraging to see so many vaccines
proving and going into development. WHO is working tirelessly with partners to develop, manufacture and deploy safe and effective vaccines.   Safe and
effective vaccines are a game-changing tool: but for the foreseeable future we must continue wearing masks, cleaning our hands, ensuring good
ventilation indoors, physically distancing and avoiding crowds.   Being vaccinated does not mean that we can throw caution to the wind and put
ourselves and others at risk, particularly because research is still ongoing into how much vaccines protect not only against disease but also against
infection and transmission.  See WHO’s landscape of COVID-19 vaccine candidates for the latest information on vaccines in clinical and pre-clinical
development, generally updated twice a week. WHO’s COVID-19 dashboard, updated daily, also features the number

Musk tweeted that his electric vehicle-making company tesla will not accept payments in bitcoin because of environmental concerns. He also said that
the company was working with developers of dogecoin to improve system transaction efficiency. The world's largest cryptocurrency hit a two-month low,
while doge coin rallied by about 20 percent. Musk has in recent months often tweeted in support of crypto, but rarely for bitcoin.

# **Summarization with T5**

In [9]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer
summary_model = T5ForConditionalGeneration.from_pretrained('t5-base')
summary_tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
summary_model = summary_model.to(device)


Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

time: 48.8 s (started: 2022-04-27 14:49:37 +00:00)


In [10]:
import random
import numpy as np

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

time: 5.76 ms (started: 2022-04-27 14:50:25 +00:00)


In [25]:
import nltk
nltk.download('punkt')
nltk.download('brown')
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.tokenize import sent_tokenize

def postprocesstext (content):
  final=""
  for sent in sent_tokenize(content):
    sent = sent.capitalize()
    final = final +" "+sent
  return final


def summarizer(text,model,tokenizer):
  text = text.strip().replace("\n"," ")
  text = "summarize: "+text
  # print (text)
  max_len = 512
  encoding = tokenizer.encode_plus(text,max_length=max_len, pad_to_max_length=False,truncation=True, return_tensors="pt").to(device)

  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = model.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=3,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  min_length = 100,
                                  max_length= 500)

  dec = [tokenizer.decode(ids,skip_special_tokens=True) for ids in outs]
  summary = dec[0]
  summary = postprocesstext(summary)
  summary= summary.strip()

  return summary
summar_text_list = []
for i in range(len(text)):
  summarized_text = summarizer(text[i],summary_model,summary_tokenizer)
  summar_text_list.append(summarized_text)
  print ("\noriginal Text >>")
  for wrp in wrap(text[i], 150):
    print (wrp)
  print ("\n")
  print ("Summarized Text >>")
  for wrp in wrap(summarized_text, 150):
    print (wrp)
  print ("\n")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1777: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size



original Text >>
Equitable access to safe and effective vaccines is critical to ending the COVID-19 pandemic, so it is hugely encouraging to see so many vaccines
proving and going into development. WHO is working tirelessly with partners to develop, manufacture and deploy safe and effective vaccines.   Safe and
effective vaccines are a game-changing tool: but for the foreseeable future we must continue wearing masks, cleaning our hands, ensuring good
ventilation indoors, physically distancing and avoiding crowds.   Being vaccinated does not mean that we can throw caution to the wind and put
ourselves and others at risk, particularly because research is still ongoing into how much vaccines protect not only against disease but also against
infection and transmission.  See WHO’s landscape of COVID-19 vaccine candidates for the latest information on vaccines in clinical and pre-clinical
development, generally updated twice a week. WHO’s COVID-19 dashboard, updated daily, also features the

# **Answer Span Extraction (Keywords and Noun Phrases)**

In [18]:
!pip install --quiet git+https://github.com/boudinfl/pke.git@dc4d5f21e0ffe64c4df93c46146d29d1c522476b
!pip install --quiet flashtext==2.7

     |████████████████████████████████| 235 kB 5.5 MB/s 
time: 14.8 s (started: 2022-04-27 14:59:54 +00:00)


In [19]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import pke
import traceback

def get_nouns_multipartite(content):
    out=[]
    try:
        extractor = pke.unsupervised.MultipartiteRank()
        extractor.load_document(input=content)
        #    not contain punctuation marks or stopwords as candidates.
        pos = {'PROPN','NOUN'}
        #pos = {'PROPN','NOUN'}
        stoplist = list(string.punctuation)
        stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
        stoplist += stopwords.words('english')
        extractor.candidate_selection(pos=pos, stoplist=stoplist)
        # 4. build the Multipartite graph and rank candidates using random walk,
        #    alpha controls the weight adjustment mechanism, see TopicRank for
        #    threshold/method parameters.
        extractor.candidate_weighting(alpha=1.1,
                                      threshold=0.75,
                                      method='average')
        keyphrases = extractor.get_n_best(n=15)
        

        for val in keyphrases:
            out.append(val[0])
    except:
        out = []
        traceback.print_exc()

    return out

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
time: 2.85 s (started: 2022-04-27 15:00:16 +00:00)


In [33]:
from flashtext import KeywordProcessor


def get_keywords(originaltext,summarytext):
  keywords = get_nouns_multipartite(originaltext)
  print ("keywords unsummarized: ",keywords)
  keyword_processor = KeywordProcessor()
  for keyword in keywords:
    keyword_processor.add_keyword(keyword)

  keywords_found = keyword_processor.extract_keywords(summarytext)
  keywords_found = list(set(keywords_found))
  print ("keywords_found in summarized: ",keywords_found)

  important_keywords =[]
  for keyword in keywords:
    if keyword in keywords_found:
      important_keywords.append(keyword)

  return important_keywords[:4]

imp_keywords_list = []

for i in range(len(text)):
  imp_keywords = get_keywords(text[i],summar_text_list[i])
  imp_keywords_list.append(imp_keywords)
  print (imp_keywords)


keywords unsummarized:  ['vaccines', 'dashboard', 'development', 'vaccination dashboard', 'access', 'information', 'risk', 'infection', 'others', 'transmission', 'number', 'week', 'landscape', 'game', 'paho']
keywords_found in summarized:  ['vaccines', 'infection', 'transmission']
['vaccines', 'infection', 'transmission']
keywords unsummarized:  ['capital', 'covid cases', 'traders', 'associations', 'delhi', 'posters', 'sanitisation', 'dos', 'cases', 'coronavirus infections', 'uptick', 'markets', 'masking', 'number', 'days']
keywords_found in summarized:  ['sanitisation', 'traders', 'number', 'dos', 'cases', 'markets', 'covid cases', 'posters', 'associations', 'masking', 'delhi']
['covid cases', 'traders', 'associations', 'delhi']
keywords unsummarized:  ['associations', 'ndta', 'members', 'booster doses', 'shop owners', 'traders', 'safety', 'customers', 'president', 'atul bhargava', 'new delhi traders association', 'vaccine camps', 'adults', 'markets', 'precautions']
keywords_found in 

# **Question generation with T5**

In [21]:
question_model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_squad_v1')
question_tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_squad_v1')
question_model = question_model.to(device)

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

time: 1min 40s (started: 2022-04-27 15:01:44 +00:00)


In [34]:
import csv
q=[]
a=[]
def get_question(context,answer,model,tokenizer):
  text = "context: {} answer: {}".format(context,answer)
  encoding = tokenizer.encode_plus(text,max_length=384, pad_to_max_length=False,truncation=True, return_tensors="pt").to(device)
  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = model.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=5,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  max_length=72)


  dec = [tokenizer.decode(ids,skip_special_tokens=True) for ids in outs]

  Question = dec[0].replace("question:","")
  Question= Question.strip()
  return Question

with open('data.csv', 'w', newline='') as file:
  writer = csv.writer(file)
  writer.writerow(["Context", "Question", "Answer"])
  for i in range(len(text)):
    for wrp in wrap(summar_text_list[i], 150):
      print (wrp)
    print ("\n")
    
    for answer in imp_keywords_list[i]:
      row = []
      row.append(summar_text_list[i])
      ques = get_question(summar_text_list[i],answer,question_model,question_tokenizer)
      q.append(ques)
      a.append(answer.capitalize())
      row.append(ques)
      row.append(answer.capitalize())
      print (ques)
      print (answer.capitalize())
      print ("\n")
      writer.writerow(row)


Who is working tirelessly with partners to develop, manufacture and deploy safe and effective vaccines. But for the foreseeable future we must
continue wearing masks, cleaning our hands, ensuring good ventilation indoors and physically distancing and avoiding crowds. Research is still ongoing
into how much vaccine protects not only against disease but also against infection and transmission. If you’re not vaccinated, you may not be able to
get the vaccine you need.




/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1777: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


Who is working tirelessly to develop, manufacture and deploy safe and effective?
Vaccines


Research is still ongoing into how much vaccine protects against disease?
Infection


Research is still ongoing into how much vaccine protects against disease?
Transmission


Traders' associations have resumed regular sanitisation and masking. The number of active cases has now increased to 4,168 from 601 on april 11 - the
hospitalisation rate so far has been low, accounting for less than three per cent of the total cases, health department data said. Traders’
associations across the country have put up posters of dos and don’ts, and are ensuring proper masking in the markets. If you are concerned about the
rising covid cases in delhi, please


What is rising in delhi?
Covid cases


Who has resumed regular sanitization and masking?
Traders


Who has resumed regular sanitization and masking?
Associations


In what city are covid cases rising?
Delhi


The new delhi traders association (ndta) has s

In [29]:


    
    writer.writerow([1, "Linus Torvalds", "Linux Kernel"])
    writer.writerow([2, "Tim Berners-Lee", "World Wide Web"])
    writer.writerow([3, "Guido van Rossum", "Python Programming"])

['Who is working to develop, manufacture and deploy vaccines?',
 'Who is working tirelessly to develop, manufacture and deploy vaccines?',
 'Who is working tirelessly to develop, manufacture and deploy vaccines?',
 'Where are vaccines produced?',
 'Who is concerned about the rising covid cases in delhi?',
 'What profession is concerned about the rising covid cases in Delhi?',
 "Traders' associations across the country have put up posters on what?",
 'Where are labs located in delhi?',
 'Who is responsible for the delhi traders association?',
 'Who is the president of the ndta?',
 'What does ndta stand for?',
 'Where is the ndta based?',
 'Who forced markets to follow the odd-even arrangement for several weeks?',
 'What profession is chandni chowk known for?',
 'Traders at chandni chowk have brought back what protocol?',
 'Traders at chandni chowk have brought back covid protocols from what?',
 'Who will continue its search to find out whether any new variant of coronavirus has evolved?

time: 5.92 ms (started: 2022-04-27 15:22:33 +00:00)


In [30]:
a

['State government',
 'Doctors',
 'Network',
 'Laboratories',
 'State government',
 'Doctors',
 'Network',
 'Laboratories',
 'State government',
 'Doctors',
 'Network',
 'Laboratories',
 'State government',
 'Doctors',
 'Network',
 'Laboratories',
 'State government',
 'Doctors',
 'Network',
 'Laboratories']

time: 3.36 ms (started: 2022-04-27 15:22:37 +00:00)


In [36]:
d=dict(zip(q,a))
d

{'What country is wuhan in?': 'Republic',
 'What is the population of wuhan?': 'People',
 'What new coronavirus causes Covid-19?': 'Sars',
 'Where was there a case of viral pneumonia?': 'Wuhan'}

time: 4.54 ms (started: 2022-04-27 12:20:17 +00:00)
